# 2.Fine-Tune a Model: Description: Fine-tune a pre-trained CNN model (e.g., DeepFace or any of your choice) for age detection on the UTK dataset. Description: GUI is not necessary. It will be evaluated on the basis of accuracy score.


In [1]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("jangedoo/utkface-new")
print("Path to dataset files:", path)

Path to dataset files: C:\Users\sreel\.cache\kagglehub\datasets\jangedoo\utkface-new\versions\1


In [65]:
import os

base_path = "C:/Users/sreel/.cache/kagglehub/datasets/jangedoo/utkface-new/versions/1"  # update if needed
for folder in os.listdir(base_path):
    full_path = os.path.join(base_path, folder)
    if os.path.isdir(full_path):
        img_count = len([f for f in os.listdir(full_path) if f.lower().endswith(('.jpg', '.png'))])
        print(f"{folder}: {img_count} image files")


crop_part1: 9780 image files
UTKFace: 23708 image files
utkface_aligned_cropped: 0 image files


In [71]:
import os

base_path = "C:/Users"  # Start from Users folder (can be changed if needed)

for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.lower().endswith(('.jpg', '.png')):
            print("✅ Found image at:", os.path.join(root, file))
            break
    else:
        continue
    break


✅ Found image at: C:/Users\Default\AppData\Local\Packages\Microsoft.MicrosoftEdge_8wekyb3d8bbwe\LocalState\PinnedTiles\Adobe.png


In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error



In [75]:
# Dataset path (change if needed)
dataset_path =  r"C:/Users/sreel/.cache/kagglehub/datasets/jangedoo/utkface-new/versions/1/crop_part1"


In [77]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),  # Augmentation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])



In [79]:
# Custom Dataset class
class UTKFaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(root_dir) if f.endswith('.jpg') or f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        parts = img_name.split("_")
        age = int(parts[0])  # Extract age
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(age, dtype=torch.float32)



In [81]:
# Create dataset
dataset = UTKFaceDataset(dataset_path, transform=transform)



In [83]:
# Split into train, val, test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])



In [85]:
# DataLoaders
batch_size = 16
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)



In [87]:
# Load pretrained model
model = models.resnet18(weights='IMAGENET1K_V1')  # Updated to use weights arg
for param in model.parameters():
    param.requires_grad = False  # Freeze layers



In [89]:
# Modify final layer for regression
model.fc = nn.Linear(model.fc.in_features, 1)



In [91]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [93]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-4)



In [103]:
# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs.squeeze(), labels)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}")



Epoch [1/5], Train Loss: 691.1118
Epoch [1/5], Validation Loss: 630.5422
Epoch [2/5], Train Loss: 612.3336
Epoch [2/5], Validation Loss: 597.9228
Epoch [3/5], Train Loss: 584.7786
Epoch [3/5], Validation Loss: 575.9292
Epoch [4/5], Train Loss: 556.0000
Epoch [4/5], Validation Loss: 547.6474
Epoch [5/5], Train Loss: 537.1273
Epoch [5/5], Validation Loss: 530.2068


In [105]:
# Save the trained model
torch.save(model.state_dict(), 'age_detection_model.pth')



In [107]:
# Evaluate on test set
model.eval()
test_loss = 0.0
predictions = []
true_values = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs.squeeze(), labels)
        test_loss += loss.item()
        predictions.extend(outputs.squeeze().cpu().numpy())
        true_values.extend(labels.cpu().numpy())

avg_test_loss = test_loss / len(test_loader)
mae = mean_absolute_error(true_values, predictions)
mse = mean_squared_error(true_values, predictions)

print(f"Test Loss: {avg_test_loss:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"Mean Squared Error: {mse:.4f}")


Test Loss: 523.7951
Mean Absolute Error: 19.3268
Mean Squared Error: 524.9370
